# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [ ]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

Initial configuration for the notebook to run properly.

In [1]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

We will load our settings and get the LLM service to use for the notebook.

In [2]:
from services import Service

from samples.service_settings import ServiceSettings

service_settings = ServiceSettings()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [3]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

Let's define a prompt outlining a dialogue chat bot.


In [4]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [5]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [6]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [7]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [8]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Sure! I'd be happy to help. Could you tell me a bit about what kind of books you enjoy? For example, do you prefer fiction or nonfiction, a particular genre (mystery, fantasy, romance, etc.), or books by certain authors?


Update the history with the output


In [9]:
chat_history.add_assistant_message(str(response))

In [19]:
from pprint import pprint
for msg in chat_history.messages:
    pprint(msg.to_dict())

{'content': 'You are a helpful chatbot who is good about giving book '
            'recommendations.',
 'role': 'system'}
{'content': "Sure! I'd be happy to help. Could you tell me a bit about what "
            'kind of books you enjoy? For example, do you prefer fiction or '
            'nonfiction, a particular genre (mystery, fantasy, romance, etc.), '
            'or books by certain authors?',
 'role': 'assistant'}


Keep Chatting!


In [20]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(
        chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [21]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely! Here are some great book suggestions that focus on history and philosophy related to Greece:

### History:
1. **"The Histories" by Herodotus**  
   Often called "The Father of History," Herodotus provides fascinating insights into ancient Greek history, culture, and their interactions with neighboring civilizations.

2. **"The Peloponnesian War" by Thucydides**  
   This classic work by the ancient historian Thucydides offers a detailed account of the conflict between Athens and Sparta, shedding light on politics, war, and human nature.

3. **"The Greeks: A Global History" by Roderick Beaton**  
   This modern book explores the history of Greece from antiquity to the present day, giving readers an accessible and comprehensive overview.

4. **"A Short History of Ancient Greece" by P.J. Rhodes**  
   A concise and engaging look at the major events, figures, and developm

In [22]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Sure, let me break down a couple of those books in more detail so you can get a sense of what they're about:

### History:
#### **"The Peloponnesian War" by Thucydides**  
This book is a detailed account of the war between Athens and Sparta in the 5th century BCE. Thucydides, who was an Athenian general, provides an analytical approach to history, focusing on the political and military strategies, as well as the human motivations behind the conflict. His work is considered one of the first examples of "scientific history" because he avoids mythology and relies on facts, eyewitness accounts, and rational analysis.

#### **"The Greeks: A Global History" by Roderick Beaton**  
This modern book offers a sweeping narrative of Greek history, ranging from the Mycenaean civilization to modern Greece. It explores how Greek culture influenced the world, from philosophy and art to politics and science. The book is written for general reade

In [23]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: If you read any of the books I suggested, you'll gain fascinating insights into different aspects of Greek history, culture, and thought. Here's a breakdown of what you might learn:

### **History-Focused Books:**
#### **"The Peloponnesian War" by Thucydides**  
- You'll learn about the political and military landscape of ancient Greece during the 5th century BCE, specifically the conflict between Athens and Sparta.  
- You'll gain an understanding of the causes of war, strategies used by both sides, and the devastating consequences for the Greek city-states.  
- Thucydides offers timeless lessons about power, ambition, human nature, and the cyclical nature of history, making the book relevant even today.  

#### **"The Greeks: A Global History" by Roderick Beaton**  
- You'll explore the full scope of Greek history, from the Mycenaean period (c. 1600 BCE) to modern Greece, including key events like the 

In [24]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are **more books** for you to explore that delve into Greek history and philosophy, as well as its mythology and culture:

---

### **History:**
1. **"The Rise of Athens: The Story of the World's Greatest Civilization" by Anthony Everitt**  
   This engaging book chronicles the history of Athens, from its beginnings to its rise as the intellectual and cultural center of ancient Greece. It examines its contributions to democracy, philosophy, and the arts.

2. **"Alexander the Great" by Robin Lane Fox**  
   Learn about the life and legacy of Alexander the Great, one of history's most famous military leaders. This biography explores his conquests, ambitions, and the impact of his empire on the ancient world.

3. **"Persian Fire: The First World Empire and the Battle for the West" by Tom Holland**  
   This book focuses on the Persian Wars (490–479 BCE), a pivotal moment in Greek history. It explo

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [27]:
for msg in chat_history:
    pprint(msg.to_dict())

{'content': 'You are a helpful chatbot who is good about giving book '
            'recommendations.',
 'role': 'system'}
{'content': "Sure! I'd be happy to help. Could you tell me a bit about what "
            'kind of books you enjoy? For example, do you prefer fiction or '
            'nonfiction, a particular genre (mystery, fantasy, romance, etc.), '
            'or books by certain authors?',
 'role': 'assistant'}
{'content': "I love history and philosophy, I'd like to learn something new "
            'about Greece, any suggestion?',
 'role': 'user'}
{'content': 'Absolutely! Here are some great book suggestions that focus on '
            'history and philosophy related to Greece:\n'
            '\n'
            '### History:\n'
            '1. **"The Histories" by Herodotus**  \n'
            '   Often called "The Father of History," Herodotus provides '
            'fascinating insights into ancient Greek history, culture, and '
            'their interactions with neighborin